In [1]:
import vision as vis
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion
import tdmclient.notebook
await tdmclient.notebook.start()

COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [ ]:
# Main -> state machine

state = COMPUTE
obstacle = False
path_calculated = False

cap = cv.VideoCapture(0)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while(1):
    time.sleep(0.2)
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
            cv.imshow("environment", im3)
            cv.waitKey(0)
            captured, img_travel = cap.read()
            cropped, cropped_travel = vis.crop_map(img_travel)
            last_pos = robot_pos
            success, robot_pos = pos.get_robot_position(img)
            if not success:
                robot_pos = last_pos
            err = motion.get_error(robot_pos,path_arch[1])
            motor_left, motor_right = motion.speed_control(err)
            motor_left_target = motor_left
            motor_right_target = motor_right
    cv.imshow("environment",im3)
    cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

C:\Users\sylpe\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Start (476, 186)
Goal (426, 156)
Vertices [[(209, 220), (354, 273), (295, 389)], [(430, 194), (498, 295), (379, 372), (314, 272)], [(142, 115), (195, 226), (67, 286), (14, 176)], [(325, 38), (389, 45), (376, 261), (321, 256)]]
list neighbours [[(4, 46.69047011971501), (5, 111.19802156513397), (13, 165.6804152578089), (16, 58.309518948453004)], [(3, 189.62331080328704), (7, 117.17081547894082), (8, 124.55520864259351), (9, 15.231546211727817), (10, 156.58863304850706), (12, 215.82400237230334), (15, 117.64352935882194)], [(5, 145.67086187704115), (6, 102.1077861869505), (7, 40.01249804748511), (14, 25.059928172283335)], [(1, 189.62331080328704), (6, 85.70297544426332), (7, 118.53269591129698), (8, 313.82319863260585), (9, 191.23022773609824), (10, 250.185930859431)], [(0, 46.69047011971501), (5, 121.75795661885921), (13, 154.53802121160993), (14, 86.05230967266364), (16, 38.2099463490856)], [(0, 111.19802156513397), (2, 145.67086187704115), (4, 121.75795661885921), (6, 141.7391971192161

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    